In [1]:
import os
import numpy as np
import pandas as pd
from taigapy import create_taiga_client_v3  
from taigapy.client_v3 import UploadedFile, LocalFormat, TaigaReference
tc = create_taiga_client_v3()
import chronos

import matplotlib.pyplot as plt
import seaborn as sns

from cds import inference
from cds import utilities as ut

pd.set_option("display.max_rows", 500)

2025-07-31 23:04:01.113310: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-07-31 23:04:01.182715: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


In [2]:
TAIGA_DATASET = 'rasscreen07212025-0b0e'
LIB_DIRECTORY = "/home/ubuntu/paralog/ras_screens/analysis/RASScreen07212025"

# Load data

In [3]:
RASGeneEffectScreenLevel = tc.get(name=TAIGA_DATASET, file='RASGeneEffectScreenLevel')  # download_to_cache for raw
RASGeneEffect = RASGeneEffectScreenLevel

RASGeneMap = tc.get(name='ras-library-files-35fa', file='RASLibraryGeneMap')
RASSequenceMap = tc.get(name=TAIGA_DATASET, file='RASSequenceMap')
RASSequenceMap = RASSequenceMap.set_index("SequenceName")
RASScreenMap = tc.get(name=TAIGA_DATASET, file='RASScreenMap')

control_genes = {
    'essentials': RASGeneMap.loc[RASGeneMap.IsCommonEssentials, "GuideTargetSymbol"],
    'nonessentials' : RASGeneMap.loc[RASGeneMap.IsNonessentials, "GuideTargetSymbol"]
}

No dataset version provided. Using version 20.
No dataset version provided. Using version 15.
No dataset version provided. Using version 20.
No dataset version provided. Using version 20.


[                      ]  0% |   0.0 s/B |   0.0 B /   2.5 KiB | ETA:  --:--:--
[##################]100% |   2.7 MiB/s |   2.5 KiB /   2.5 KiB | ETA:  00:00:00


In [4]:
RASSequenceMap

,Unnamed: 0,Cas,Days,Screen,CellLine,ScreenType,Replicates,Treatment,Source,NNMD,...,OncotreeLineage,OncotreePrimaryDisease,OncotreeSubtype,OncotreeCode,SangerModelID,COSMICID,CRISPR,CopyNumber,Expression,LOH
SequenceName,,,,,,,,,,,,,,,,,,,,,
PACADD188-Cas9VY-Ras-RepNA-P5(D25),0,Cas9VY,25,PACADD188,PACADD188,Parental,RepA,NoTreatment,Sellers,-7.685994,...,Pancreas,Pancreatic Adenocarcinoma,Pancreatic Adenocarcinoma,PAAD,None,NaN,True,True,True,True
PANC0203_500x-Cas9VY-RAS-RepNA-P5(D17),1,Cas9VY,17,PANC0203_500x_D17,PANC0203,Parental,RepA,NoTreatment,Sellers,-4.779225,...,Pancreas,Pancreatic Adenocarcinoma,Pancreatic Adenocarcinoma,PAAD,SIDM01139,1298475.0,True,True,True,False
PANC0203_500x-Cas9VY-RAS-RepNA-P5(D13),2,Cas9VY,13,PANC0203_500x_D13,PANC0203,Parental,RepA,NoTreatment,Sellers,-3.220243,...,Pancreas,Pancreatic Adenocarcinoma,Pancreatic Adenocarcinoma,PAAD,SIDM01139,1298475.0,True,True,True,False
PANC0203_500x-Cas9VY-RAS-RepNA-P4(D5),3,Cas9VY,5,PANC0203_500x_D5,PANC0203,Parental,RepA,NoTreatment,Sellers,-0.204920,...,Pancreas,Pancreatic Adenocarcinoma,Pancreatic Adenocarcinoma,PAAD,SIDM01139,1298475.0,True,True,True,False
PANC0203_500x-Cas9VY-RAS-RepNA-P4(D9),4,Cas9VY,9,PANC0203_500x_D9,PANC0203,Parental,RepA,NoTreatment,Sellers,-1.215390,...,Pancreas,Pancreatic Adenocarcinoma,Pancreatic Adenocarcinoma,PAAD,SIDM01139,1298475.0,True,True,True,False
PACADD165_250x-Cas9VY-RAS-RepNA-P4(D21),5,Cas9VY,21,PACADD165,PACADD165,Parental,RepA,NoTreatment,Sellers,-7.706549,...,Pancreas,Pancreatic Adenocarcinoma,Pancreatic Adenocarcinoma,PAAD,None,NaN,True,True,True,False
KP363T_250x-Cas9VY-RAS-RepNA-P4(D21),6,Cas9VY,21,KP363T,KP363T,Parental,RepA,NoTreatment,Sellers,-9.250705,...,Bowel,Colorectal Adenocarcinoma,Colon Adenocarcinoma,COAD,None,NaN,True,True,True,True
SNU1079_250x-Cas9VY-RAS-RepNA-P5(D25),7,Cas9VY,25,SNU1079,SNU1079,Parental,RepA,NoTreatment,Sellers,-0.560421,...,Biliary Tract,Intraductal Papillary Neoplasm of the Bile Duct,Intrahepatic Cholangiocarcinoma,IHCH,SIDM00164,NaN,True,True,True,True
COLO678_250x-Cas9VY-RAS-RepNA-P5(D25),8,Cas9VY,25,COLO678,COLO678,Parental,RepA,NoTreatment,Sellers,-5.404073,...,Bowel,Colorectal Adenocarcinoma,Colon Adenocarcinoma,COAD,SIDM00957,910689.0,True,True,True,True


In [5]:
OmicsExpressionProteinCodingGenesTPMLogp1 = tc.get(name='internal-24q4-8c04', file='OmicsExpressionProteinCodingGenesTPMLogp1')
# Remove entrez Id
OmicsExpressionProteinCodingGenesTPMLogp1.columns = OmicsExpressionProteinCodingGenesTPMLogp1.columns.str.split(" ").str[0]

# Align expression using screen name
all_screens = pd.DataFrame({'ScreenName' : RASGeneEffect.index})
screen_to_model = RASSequenceMap[['Screen', 'ModelID']].reset_index(drop=True).drop_duplicates()
screen_to_model = dict(zip(screen_to_model.Screen, screen_to_model.ModelID))
all_screens['ModelID'] = all_screens['ScreenName'].map(screen_to_model)
all_screens['HavingOmicsExpr'] = all_screens['ModelID'].isin(OmicsExpressionProteinCodingGenesTPMLogp1.index)


No dataset version provided. Using version 120.


In [6]:
all_screens.loc[~all_screens['HavingOmicsExpr']]

,ScreenName,ModelID,HavingOmicsExpr
1,COLO205,ACH-001039,False


In [7]:
all_screens_used = all_screens.loc[ (~all_screens['ModelID'].isna()) & (all_screens['HavingOmicsExpr'])]
gene_effect_used = RASGeneEffect.loc[all_screens_used['ScreenName']]
# TO DO: don't skip gene effect without expression, use non-essential instead
expr_used = OmicsExpressionProteinCodingGenesTPMLogp1.loc[all_screens_used['ModelID']]
expr_used.index = all_screens_used['ScreenName']

In [8]:
expr_used.to_csv(os.path.join(LIB_DIRECTORY, "RASExpressionProfile.csv"), index=True)

# Gene dependency inference

In [9]:
def melt_and_annotate(gene_effect, expression, essentials, nonessentials, expression_cutoff):
    '''
        Produces a melted DataFrame with annotations necessary for assigning dependency probabilities.

        Parameters:
            gene_effect - matrix with genes as columns
            expression - matrix of TPM expression values with genes as columns
        Returns:
            melted (table) - annotations
    '''
        #copy gene effect and melt
    gene_effect = gene_effect.copy()
    melted = ut.index_melt(gene_effect)
        #rename columns
    melted.columns = ['lines', 'genes', 'score']
        #label if essential/nonessential
    melted['essential'] = melted.genes.isin(essentials)
    melted['nonessential'] = melted.genes.isin(nonessentials)
    #melt the expression data
    melt_exp = ut.index_melt(expression.reindex(
            index=gene_effect.index, columns=gene_effect.columns
    ))
        #annotate unexpressed
    melted['unexpressed'] = melt_exp.value < expression_cutoff
   
    #check means/medians of various distribitions
    mu = melted[melted.unexpressed].score.mean()

    mu2 = melted[~melted.unexpressed].score.mean()
        
    mu3 = melted[melted.essential].score.mean()
    mu4 = melted[melted.nonessential].score.median()
   
    melted.unexpressed.fillna(False, inplace=True)
    melted = melted[~melted.score.isnull()]
    return melted

def infer(data, initial_lambdas=[.85, .15], smoothing=.05, p_smoothing=.15):
    '''
        For each line, assign each gene score in that line a probability that the gene is a real dependency

        Parameters:
            data (table) - melted gene frame with columns
                'lines': cell line name
                'genes': gene name
                'score': chronos/ceres score (scaled)
                'essential': Boolean whether the gene is identified essential (all lines)
                'unexpressed': Boolean whether the gene is expressed in that cell line
                'nonessential': Boolean whether the gene is identified nonessential (all lines)
            initial_lambdas (list) - list of two float summing to 1;
                best initial guess for how much of the distribution is null vs real dependency
            smoothing, p_smoothing (float) - kernel parameters for distribution estimation, see cds.inference.probability_2class

        Returns:
            gene_dependency (matrix) - probabilities of each gene being a dependency in each cell line
            gene_fdr (matrix) - FDR values (see API)
    '''
    
    data['p_dependent'] = np.nan
    data['FDR'] = np.nan

        #assign probabilities per line
    for line in data.lines.unique():
        inference.mixture_model.assign_probabilities(
                data, line, initial_lambdas, fit_gaussian=False,
                smoothing=smoothing, p_smoothing=p_smoothing
        )
    data.sort_values(by='p_dependent', ascending=False, inplace=True)

        #calculate FDRs
    for line in pd.unique(data.lines):
        vals = np.cumsum(
                1 - data[data.lines == line].p_dependent
            )
        data.loc[vals.index, 'FDR'] = vals*1.0/np.arange(1, len(vals)+1)

    #check means
    unexpr_mean = data.p_dependent[data.unexpressed].mean()
  
    noness_mean = data.p_dependent[data.nonessential].mean()
     
    ess_mean = data.p_dependent[data.essential].mean()
       

        #convert into matrices
    gene_dependency = data.pivot(index='lines', columns='genes', values='p_dependent')
    gene_dependency.index.name = 'line'
    gene_fdr = data.pivot(index='lines', columns='genes', values='FDR')
    gene_fdr.index.name = 'line'
    return gene_dependency, gene_fdr

In [10]:
melted = melt_and_annotate(gene_effect=RASGeneEffect, expression=expr_used, 
                  essentials=control_genes['essentials'], nonessentials=control_genes['nonessentials'], expression_cutoff=0.2)



/tmp/ipykernel_25982/535025636.py:34: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  melted.unexpressed.fillna(False, inplace=True)


In [12]:
# Infer gene dependency and fdr
gene_dependency, gene_fdr = infer(data=melted, initial_lambdas=[.85, .15], smoothing=.05, p_smoothing=.15)

Converged at cost -9.689180 with 8 iterations
Converged at cost -9.717794 with 10 iterations
Converged at cost -9.379061 with 12 iterations
Converged at cost -9.315723 with 10 iterations
Converged at cost -9.443663 with 9 iterations
Converged at cost -9.560617 with 8 iterations
Converged at cost -9.463192 with 9 iterations
Converged at cost -9.455679 with 8 iterations
Converged at cost -9.431447 with 14 iterations
Converged at cost -9.614697 with 12 iterations
Converged at cost -9.469564 with 16 iterations
Converged at cost -9.545843 with 13 iterations
Converged at cost -9.392357 with 12 iterations
Converged at cost -9.917140 with 6 iterations
Converged at cost -9.624895 with 9 iterations


In [13]:
# Fixed index and column names
gene_dependency.index.name = 'ScreenName'
gene_dependency.columns.name = 'GuideTargetSymbol'

gene_fdr.index.name = 'ScreenName'
gene_fdr.columns.name = 'GuideTargetSymbol'

In [14]:
gene_dependency.to_csv(os.path.join(LIB_DIRECTORY, "RASGeneDependency.csv"))
gene_fdr.to_csv(os.path.join(LIB_DIRECTORY, "RASGeneDependencyFdr.csv"))

In [15]:
import os
from taigapy import create_taiga_client_v3
from taigapy.client_v3 import UploadedFile, LocalFormat, TaigaReference
tc = create_taiga_client_v3()
version = tc.update_dataset(TAIGA_DATASET,  
                            'Uploading gene dependency results',
                            additions=[
    UploadedFile('RASGeneDependency', 
                 local_path=os.path.join(LIB_DIRECTORY, "RASGeneDependency.csv"), 
                 format=LocalFormat.CSV_MATRIX),
    UploadedFile('RASGeneDependencyFdr', 
                 local_path=os.path.join(LIB_DIRECTORY, "RASGeneDependencyFdr.csv"), 
                 format= LocalFormat.CSV_MATRIX)    
]
)

Created temporary upload session 95948caef6af48ee89fa59ad61147876
Uploading /home/ubuntu/paralog/ras_screens/analysis/RASScreen07212025/RASGeneDependency.csv to S3
Finished uploading /home/ubuntu/paralog/ras_screens/analysis/RASScreen07212025/RASGeneDependency.csv to S3
Completed uploading /home/ubuntu/paralog/ras_screens/analysis/RASScreen07212025/RASGeneDependency.csv to S3
Added /home/ubuntu/paralog/ras_screens/analysis/RASScreen07212025/RASGeneDependency.csv to upload session 95948caef6af48ee89fa59ad61147876
Uploading /home/ubuntu/paralog/ras_screens/analysis/RASScreen07212025/RASGeneDependencyFdr.csv to S3
Finished uploading /home/ubuntu/paralog/ras_screens/analysis/RASScreen07212025/RASGeneDependencyFdr.csv to S3
Completed uploading /home/ubuntu/paralog/ras_screens/analysis/RASScreen07212025/RASGeneDependencyFdr.csv to S3
Added /home/ubuntu/paralog/ras_screens/analysis/RASScreen07212025/RASGeneDependencyFdr.csv to upload session 95948caef6af48ee89fa59ad61147876
Dataset created. A

# Find essentials

Use parental screens only

In [16]:
screen_map = RASSequenceMap[[ 'Screen', 'ModelID', 'ScreenType', 'Treatment']].reset_index(drop=True).drop_duplicates().reset_index(drop=True)

In [17]:
screen_map.rename(columns={'Screen' : 'ScreenName'}, inplace=True)

In [18]:
all_screens = pd.DataFrame({'ScreenName' : RASGeneEffect.index})
all_screens = all_screens.merge(screen_map, how='left')
parental_screens  = all_screens.loc[all_screens['ScreenType'] == 'Parental']

In [19]:
parental_screens.shape

(15, 4)

In [20]:
from scipy.stats import gaussian_kde

def find_essentials(gene_effect, percentile:float, max_thresh:float):
    '''
        Find a list of essential genes based on consistently high dependency score in large number of cell lines.

        Parameters:
            gene_effect (matrix) - genes as columns
            percentile (float) - fraction of cell lines a given gene must exceed dependency threshold 
                                    for in order to be considered panessential
            max_thresh (float) - threshold for calling essentials
        Returns:
            dependency threshold value,
            list of essential genes
    '''
        #rank genes by percentile in each line
    ranks = gene_effect.rank(axis=1, pct=True)
        #rank gene ranks by percentile in each gene
    ranks_lines = ranks.rank(axis=0, pct=True)
        #find min rank for lines with rank above percentile threshold
    gene_scores = ranks[ranks_lines > percentile].min(axis=0)
        #fit a KDE
    kde = gaussian_kde(gene_scores.dropna(), bw_method=.1)
    test_points = np.arange(.1, .8, .001)
        #get natural cutoff suggested by the data
    threshold = test_points[np.argmin(kde(test_points))]
    if threshold > max_thresh and gene_effect.shape[0] > 50:
        threshold = max_thresh  
        #select essentials
    essentials = pd.Series(gene_scores.index[gene_scores < threshold], name="Essentials")
        #check mean
    ess_mean = gene_effect[essentials].mean().mean()
    if ess_mean > -.5:
        print(f'the mean of all {len(essentials)} found essential genes' + f'({ess_mean}) is higher than expected')
    return threshold, essentials

In [21]:
threshold, essentials = find_essentials(gene_effect=RASGeneEffect.loc[parental_screens['ScreenName'],], percentile=.9, max_thresh=0.1)

In [22]:
RASEssentials = pd.DataFrame(
        {'GuideTargetSymbol':essentials}
        )
RASEssentials['GeneEffectMean'] = RASGeneEffect[essentials].mean().values
RASEssentials['GeneEffectMedian'] = RASGeneEffect[essentials].median().values
RASEssentials['GeneEffectMin'] =RASGeneEffect[essentials].min().values
RASEssentials['GeneEffectMax'] = RASGeneEffect[essentials].max().values

In [23]:
RASEssentials.set_index("GuideTargetSymbol").to_csv(os.path.join(LIB_DIRECTORY, "RASInferredEssentials.csv"))


## Annotating essentials

In [24]:
CRISPRInferredCommonEssentials = tc.get(name='internal-24q4-8c04',  file='CRISPRInferredCommonEssentials')
CRISPRInferredCommonEssentials[['Gene', 'EntrezId']] = CRISPRInferredCommonEssentials['Essentials'].str.split(' ', expand=True)
RASs = RASEssentials.loc[RASEssentials['GuideTargetSymbol'].str.contains('_')].copy()
singlegene = RASEssentials.loc[~RASEssentials['GuideTargetSymbol'].str.contains('_')].copy()
singlegene.loc[:, 'isGeneCommoneEssentialInAVANA'] = singlegene['GuideTargetSymbol'].isin(CRISPRInferredCommonEssentials['Gene'])
RASs[['GeneA', 'GeneB']] = RASs['GuideTargetSymbol'].str.split('_', expand=True)
RASs.loc[:, 'isGeneACommonEssentialInRAS'] =RASs['GeneA'].isin(singlegene['GuideTargetSymbol'])
RASs.loc[:, 'isGeneBCommonEssentialInRAS'] =RASs['GeneB'].isin(singlegene['GuideTargetSymbol'])
# Depmap essentials

RASs.loc[:, 'isGeneACommonEssentialInAVANA'] =RASs['GeneA'].isin(CRISPRInferredCommonEssentials['Gene'])
RASs.loc[:, 'isGeneBCommonEssentialInAVANA'] =RASs['GeneB'].isin(CRISPRInferredCommonEssentials['Gene'])

No dataset version provided. Using version 120.


In [25]:
RASs.set_index('GuideTargetSymbol').to_csv(os.path.join(LIB_DIRECTORY, "RASInferredEssentialPairs.csv"))
singlegene.set_index('GuideTargetSymbol').to_csv(os.path.join(LIB_DIRECTORY, "RASInferredEssentialGenes.csv"))


## Upload to taiga

In [26]:
import os
from taigapy import create_taiga_client_v3
from taigapy.client_v3 import UploadedFile, LocalFormat, TaigaReference
tc = create_taiga_client_v3()
version = tc.update_dataset(TAIGA_DATASET,  
                            'Uploading essentials',
                            additions=[
    UploadedFile('RASInferredEssentials', 
                 local_path=os.path.join(LIB_DIRECTORY, "RASInferredEssentials.csv"), 
                 format=LocalFormat.CSV_TABLE),
    UploadedFile('RASInferredEssentialPairs', 
                 local_path=os.path.join(LIB_DIRECTORY, "RASInferredEssentialPairs.csv"), 
                 format= LocalFormat.CSV_TABLE),
     UploadedFile('RASInferredEssentialGenes', 
                 local_path=os.path.join(LIB_DIRECTORY, "RASInferredEssentialGenes.csv"), 
                 format= LocalFormat.CSV_TABLE),
        
]
)

Created temporary upload session 66fb6eba2683434aae07420cee8ae784
Uploading /home/ubuntu/paralog/ras_screens/analysis/RASScreen07212025/RASInferredEssentials.csv to S3
Finished uploading /home/ubuntu/paralog/ras_screens/analysis/RASScreen07212025/RASInferredEssentials.csv to S3
Completed uploading /home/ubuntu/paralog/ras_screens/analysis/RASScreen07212025/RASInferredEssentials.csv to S3
Added /home/ubuntu/paralog/ras_screens/analysis/RASScreen07212025/RASInferredEssentials.csv to upload session 66fb6eba2683434aae07420cee8ae784
Uploading /home/ubuntu/paralog/ras_screens/analysis/RASScreen07212025/RASInferredEssentialPairs.csv to S3
Finished uploading /home/ubuntu/paralog/ras_screens/analysis/RASScreen07212025/RASInferredEssentialPairs.csv to S3
Completed uploading /home/ubuntu/paralog/ras_screens/analysis/RASScreen07212025/RASInferredEssentialPairs.csv to S3
Added /home/ubuntu/paralog/ras_screens/analysis/RASScreen07212025/RASInferredEssentialPairs.csv to upload session 66fb6eba2683434

# RAS Synergy

# TODO:  be a function to regonize screen names

In [27]:
# For safty, reload gene effect
RASGeneEffectScreenLevel = tc.get(name=TAIGA_DATASET, file='RASGeneEffectScreenLevel')  # download_to_cache for raw
RASGeneEffect = RASGeneEffectScreenLevel

RASScreenMap = tc.get(name=TAIGA_DATASET,  file='RASScreenMap')
RASScreenMap.loc[RASScreenMap['ModelID'].isna()]

No dataset version provided. Using version 22.
No dataset version provided. Using version 22.


,Unnamed: 0,Screen,CellLine,ScreenType,Treatment,Source,CRISPR,CopyNumber,Expression,LOH,...,StrippedCellLineName,CCLEName,DepmapModelType,OncotreeLineage,OncotreePrimaryDisease,OncotreeSubtype,OncotreeCode,SangerModelID,COSMICID,SpearmanAVANA_y
18,23,pDNA,pDNA,pDNA,NoTreatment,Sellers,False,False,False,False,...,None,None,None,None,None,None,None,None,NaN,NaN


In [28]:
OmicsExpressionProteinCodingGenesTPMLogp1 = tc.get(name='internal-24q4-8c04',  file='OmicsExpressionProteinCodingGenesTPMLogp1')
# Remove entrez Id
OmicsExpressionProteinCodingGenesTPMLogp1.columns = OmicsExpressionProteinCodingGenesTPMLogp1.columns.str.split(" ").str[0]

No dataset version provided. Using version 120.


In [29]:
expr_used = ut.index_melt(OmicsExpressionProteinCodingGenesTPMLogp1).rename(columns = {'index': 'ModelID', 'variable': 'gene', 'value': 'expression'})
expr_used = expr_used.merge(RASScreenMap.loc[:, ['Screen', 'ModelID']], left_on='ModelID', right_on='ModelID', how='left')
expr_used = expr_used.loc[~expr_used['Screen'].isna()]
gene_effect = RASGeneEffect.copy()

melted = ut.index_melt(gene_effect)
#rename columns
melted.columns = ['ScreenName', 'GuideTargetSymbol', 'Chronos']
melted_pairs = melted.loc[melted['GuideTargetSymbol'].str.contains("_")]
melted_pairs[['GeneA', 'GeneB']] = melted_pairs['GuideTargetSymbol'].str.split("_", expand=True)
melted_pairs = melted_pairs.rename(columns={'GuideTargetSymbol': 'Pairs', 'Chronos': 'ChronosPairs'})
melted_pairs = melted_pairs.loc[~melted_pairs['Pairs'].isin(['AAVS1_AAVS1', 'AAVS1_chr2', 'AAVS1_nonTarget', 'chr2_chr2', 'nonTarget_nonTarget'])]
control_genes = {
    'essentials': RASGeneMap.loc[RASGeneMap.IsCommonEssentials, "GuideTargetSymbol"],
    'nonessentials' : RASGeneMap.loc[RASGeneMap.IsNonessentials, "GuideTargetSymbol"]
}
essentials = control_genes['essentials']
nonessentials = control_genes['nonessentials']
melted_single_genes = melted.loc[~melted['GuideTargetSymbol'].str.contains("_")]
melted_single_genes['essential'] = melted_single_genes['GuideTargetSymbol'].isin(essentials)
melted_single_genes['nonessential'] = melted_single_genes['GuideTargetSymbol'].isin(nonessentials)


/tmp/ipykernel_25982/1182271653.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  melted_pairs[['GeneA', 'GeneB']] = melted_pairs['GuideTargetSymbol'].str.split("_", expand=True)
/tmp/ipykernel_25982/1182271653.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  melted_pairs[['GeneA', 'GeneB']] = melted_pairs['GuideTargetSymbol'].str.split("_", expand=True)
/tmp/ipykernel_25982/1182271653.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[ro

In [30]:
# Merging
melted_pairs = melted_pairs\
    .merge(melted_single_genes, left_on=['ScreenName', 'GeneA'], right_on=['ScreenName', 'GuideTargetSymbol'], how='left')\
    .drop(columns=['GuideTargetSymbol'])\
    .rename(columns={'Chronos': 'ChronosGeneA', 'essential': 'IsGeneAEssential', 'nonessential':'IsGeneAnonessential'})\
    .merge(melted_single_genes, left_on=['ScreenName', 'GeneB'], right_on=['ScreenName', 'GuideTargetSymbol'], how='left')\
    .drop(columns=['GuideTargetSymbol'])\
    .rename(columns={'Chronos': 'ChronosGeneB', 'essential': 'IsGeneBEssential', 'nonessential':'IsGeneBnonessential'})


In [31]:
expr_used.rename(columns={'Screen': 'ScreenName'}, inplace=True)

In [32]:
melted_pairs = melted_pairs\
    .merge(expr_used, left_on=['ScreenName', 'GeneA'], right_on=['ScreenName', 'gene'], how='left')\
    .drop(columns=['gene', 'ModelID'])\
    .rename(columns={'expression': 'ExpressionGeneA'})\
    .merge(expr_used, left_on=['ScreenName', 'GeneB'], right_on=['ScreenName', 'gene'], how='left')\
    .drop(columns=['gene', 'ModelID'])\
    .rename(columns={'expression': 'ExpressionGeneB'})

In [33]:
expr_cutoff = 0.2
melted_pairs['ExpressionType'] = None

melted_pairs.loc[ 
    (melted_pairs['ExpressionGeneA'] < expr_cutoff) & (~melted_pairs['ExpressionGeneA'].isna()) &  (melted_pairs['ExpressionGeneB'] < expr_cutoff) & (~melted_pairs['ExpressionGeneB'].isna()), 'ExpressionType'
] = 'non-expressed'
melted_pairs.loc[ 
    (melted_pairs['ExpressionGeneA'] >= expr_cutoff) & (~melted_pairs['ExpressionGeneA'].isna()) &  (melted_pairs['ExpressionGeneB'] < expr_cutoff) & (~melted_pairs['ExpressionGeneB'].isna()), 'ExpressionType'
] = 'uni-expressed'
melted_pairs.loc[ 
    (melted_pairs['ExpressionGeneA'] < expr_cutoff) & (~melted_pairs['ExpressionGeneA'].isna()) &  (melted_pairs['ExpressionGeneB'] >= expr_cutoff) & (~melted_pairs['ExpressionGeneB'].isna()), 'ExpressionType'
] = 'uni-expressed'
melted_pairs.loc[ 
    (melted_pairs['ExpressionGeneA'] > expr_cutoff) & (~melted_pairs['ExpressionGeneA'].isna()) &  (melted_pairs['ExpressionGeneB'] > expr_cutoff) & (~melted_pairs['ExpressionGeneB'].isna()), 'ExpressionType'
] = 'co-expressed'

RASGeneDependencyFdr = tc.get(name=TAIGA_DATASET, file='RASGeneDependencyFdr')

melted = ut.index_melt(RASGeneDependencyFdr).rename(columns = {'index' : 'ScreenName', 'variable': 'GuideTargetSymbol', 'value' : 'GeneDepFdr'})
# Join gene dependence
melted_pairs = melted_pairs\
    .merge(melted, left_on=['ScreenName', 'Pairs'], right_on=['ScreenName', 'GuideTargetSymbol'], how='left')\
    .rename(columns={'GeneDepFdr': 'PairDepFdr'})\
    .merge(melted, left_on=['ScreenName', 'GeneA'], right_on=['ScreenName', 'GuideTargetSymbol'], how='left')\
    .rename(columns={'GeneDepFdr': 'GeneADepFdr'})\
    .merge(melted, left_on=['ScreenName', 'GeneB'], right_on=['ScreenName', 'GuideTargetSymbol'], how='left')\
    .rename(columns={'GeneDepFdr': 'GeneBDepFdr'})\
    .drop(['GuideTargetSymbol_x', 'GuideTargetSymbol_y', 'GuideTargetSymbol'], axis=1)



melted_pairs.loc[:, 'minAB'] = np.minimum(melted_pairs['ChronosGeneA'], melted_pairs['ChronosGeneB'])
melted_pairs.loc[:, 'minABfdr'] = np.where(melted_pairs['ChronosGeneA'] < melted_pairs['ChronosGeneB'],melted_pairs['GeneADepFdr'],melted_pairs['GeneBDepFdr'])

melted_pairs.loc[:, 'CombinatoryScore'] = melted_pairs['ChronosPairs'] - np.minimum(melted_pairs['ChronosGeneA'], melted_pairs['ChronosGeneB'])
melted_pairs.loc[:, 'SynergyScore'] = melted_pairs['ChronosPairs'] - (melted_pairs['ChronosGeneA'] +  melted_pairs['ChronosGeneB'])

No dataset version provided. Using version 22.


[                      ]  0% |   0.0 s/B |   0.0 B /   1.7 MiB | ETA:  --:--:--
[##################]100% |  64.1 MiB/s |   1.7 MiB /   1.7 MiB | Time:  0:00:00


In [34]:
# Add common essential containing pairs
CRISPRInferredCommonEssentials = tc.get(name='internal-24q4-8c04',  file='CRISPRInferredCommonEssentials')
CRISPRInferredCommonEssentials[['Gene', 'EntrezId']] = CRISPRInferredCommonEssentials['Essentials'].str.split(' ', expand=True)
melted_pairs['IsEssentialGeneA'] = melted_pairs['GeneA'].isin(CRISPRInferredCommonEssentials['Gene'])
melted_pairs['IsEssentialGeneB'] = melted_pairs['GeneB'].isin(CRISPRInferredCommonEssentials['Gene'])
melted_pairs['IncludingEssential'] = melted_pairs['IsEssentialGeneA'] | melted_pairs['IsEssentialGeneB']

No dataset version provided. Using version 120.


In [35]:
RASScreenMap.rename(columns={'Screen' : 'ScreenName'}, inplace=True)

In [36]:
# Mark cell lines withtout ModelID or no expression data
diff_models = RASScreenMap.loc[
    (RASScreenMap['ModelID'].isna()) | ~(RASScreenMap['ModelID'].isin(OmicsExpressionProteinCodingGenesTPMLogp1.index)),]


In [37]:
from scipy.stats import gaussian_kde

from statsmodels.stats.multitest import fdrcorrection

for line in melted_pairs['ScreenName'].unique():
    if line in diff_models['ScreenName']:
        continue
    cur_line = melted_pairs.loc[(melted_pairs['ScreenName'] == line)]
    null_model1 = cur_line.loc[ (cur_line['ExpressionType'] != 'co-expressed')   ].copy()
    null_model2 = cur_line.loc[ (cur_line['IncludingEssential'] )].copy()
    # 分别对两部分数据使用 KDE 拟合

    # Remove NaN and inf values
    clean_scores1 = null_model1['CombinatoryScore']
    clean_scores1 = clean_scores1[np.isfinite(clean_scores1)]

    clean_scores2 = null_model2['CombinatoryScore']
    clean_scores2 = clean_scores2[np.isfinite(clean_scores2)]

    # KDE
    kde1 = gaussian_kde(clean_scores1)
    kde2 = gaussian_kde(clean_scores2)

    
    #kde = gaussian_kde(null_model['CombinatoryScore'])
    n_samples = 100000
    null_model_combin1 =  kde1.resample(n_samples).flatten()
    null_model_combin2 =  kde2.resample(n_samples).flatten()
    print("Number of pairs in null model1 :", null_model1.shape[0])
    print("Number of pairs in null model2 :", null_model2.shape[0])

    
    
    
    #all_data = cur_line.loc[ (cur_line['ExpressionType'] == 'co-expressed')   ].copy()
    all_data = cur_line.copy()

    # Better sample classification
    all_data1 = all_data.loc[all_data['minAB'] >= -1].copy()
    # Essential
    all_data2 = all_data.loc[all_data['minAB'] < -1].copy()
    
    print("Number of test in current model :", all_data.shape[0])

    all_data1['CombinatoryScorePVal'] = ut.empirical_pvalue(observed= all_data1['CombinatoryScore'] , null = null_model_combin1,direction=-1)
    all_data2['CombinatoryScorePVal'] = ut.empirical_pvalue(observed= all_data2['CombinatoryScore'] , null = null_model_combin2,direction=-1)
    reject, fdr_pvalues  = fdrcorrection(all_data1['CombinatoryScorePVal'], alpha=0.05, method='indep')
    all_data1['CombinatoryScoreFdr'] = fdr_pvalues
    reject, fdr_pvalues  = fdrcorrection(all_data2['CombinatoryScorePVal'], alpha=0.05, method='indep')
    all_data2['CombinatoryScoreFdr'] = fdr_pvalues

    all_data.loc[all_data2.index, 'CombinatoryScorePVal'] = all_data2['CombinatoryScorePVal']
    all_data.loc[all_data1.index, 'CombinatoryScorePVal'] = all_data1['CombinatoryScorePVal']
    all_data.loc[all_data2.index, 'CombinatoryScoreFdr'] = all_data2['CombinatoryScoreFdr']
    all_data.loc[all_data1.index, 'CombinatoryScoreFdr'] = all_data1['CombinatoryScoreFdr']
    
   
    
     # 分别对两部分数据使用 KDE 拟合

    # Remove NaN and inf values
    clean_scores_syn1 = null_model1['SynergyScore']
    clean_scores_syn1 = clean_scores_syn1[np.isfinite(clean_scores_syn1)]

    clean_scores_syn2 = null_model2['SynergyScore']
    clean_scores_syn2 = clean_scores_syn2[np.isfinite(clean_scores_syn2)]

    # KDE
    kde1 = gaussian_kde(clean_scores_syn1)
    kde2 = gaussian_kde(clean_scores_syn2)

    
    #kde = gaussian_kde(null_model['CombinatoryScore'])
    n_samples = 100000
    null_model_syn1 =  kde1.resample(n_samples).flatten()
    null_model_syn2 =  kde2.resample(n_samples).flatten()
  
   
    all_data1['SynergyScorePVal'] = ut.empirical_pvalue(observed= all_data1['SynergyScore'] , null = null_model_combin1,direction=-1)
    all_data2['SynergyScorePVal'] = ut.empirical_pvalue(observed= all_data2['SynergyScore'] , null = null_model_combin2,direction=-1)
    
    all_data.loc[all_data1.index, 'SynergyScorePVal'] = all_data1['SynergyScorePVal']
    all_data.loc[all_data2.index, 'SynergyScorePVal'] = all_data2['SynergyScorePVal']
    reject, fdr_pvalues  = fdrcorrection(all_data['SynergyScorePVal'], alpha=0.05, method='indep')
    all_data['SynergyScoreFdr'] = fdr_pvalues


    melted_pairs.loc[all_data.index, 'CombinatoryScorePVal'] = all_data['CombinatoryScorePVal']
    melted_pairs.loc[all_data.index, 'CombinatoryScoreFdr']  = all_data['CombinatoryScoreFdr']

    melted_pairs.loc[all_data.index, 'SynergyScorePVal'] = all_data['SynergyScorePVal']
    melted_pairs.loc[all_data.index, 'SynergyScoreFdr']  = all_data['SynergyScoreFdr']
    


Number of pairs in null model1 : 2546
Number of pairs in null model2 : 980
Number of test in current model : 10296
Number of pairs in null model1 : 10296
Number of pairs in null model2 : 980
Number of test in current model : 10296
Number of pairs in null model1 : 2793
Number of pairs in null model2 : 980
Number of test in current model : 10296
Number of pairs in null model1 : 2670
Number of pairs in null model2 : 980
Number of test in current model : 10296
Number of pairs in null model1 : 3156
Number of pairs in null model2 : 980
Number of test in current model : 10296
Number of pairs in null model1 : 2915
Number of pairs in null model2 : 980
Number of test in current model : 10296
Number of pairs in null model1 : 2793
Number of pairs in null model2 : 980
Number of test in current model : 10296
Number of pairs in null model1 : 3036
Number of pairs in null model2 : 980
Number of test in current model : 10296
Number of pairs in null model1 : 3275
Number of pairs in null model2 : 980
Numb

In [38]:
melted_pairs

,ScreenName,Pairs,ChronosPairs,GeneA,GeneB,ChronosGeneA,IsGeneAEssential,IsGeneAnonessential,ChronosGeneB,IsGeneBEssential,...,minABfdr,CombinatoryScore,SynergyScore,IsEssentialGeneA,IsEssentialGeneB,IncludingEssential,CombinatoryScorePVal,CombinatoryScoreFdr,SynergyScorePVal,SynergyScoreFdr
0,BXPC3,AKT1_AKT2,-0.542547,AKT1,AKT2,-0.173663,False,False,-0.004845,False,...,0.211245,-0.368883,-0.364038,False,False,False,0.016620,0.330214,0.017010,0.746453
1,COLO205,AKT1_AKT2,-0.602124,AKT1,AKT2,-0.002584,False,False,-0.040360,False,...,0.278071,-0.561764,-0.559180,False,False,False,0.050349,0.985395,0.050789,0.851084
2,COLO678,AKT1_AKT2,-0.129872,AKT1,AKT2,-0.098261,False,False,-0.088108,False,...,0.384455,-0.031611,0.056497,False,False,False,0.234268,0.797977,0.399736,1.000000
3,HCT15,AKT1_AKT2,-0.837725,AKT1,AKT2,-0.256233,False,False,0.135923,False,...,0.129046,-0.581492,-0.717415,False,False,False,0.005790,0.245689,0.003440,0.625149
4,HPAFII,AKT1_AKT2,-1.037252,AKT1,AKT2,-0.204400,False,False,-0.148029,False,...,0.125028,-0.832852,-0.684822,False,False,False,0.000010,0.001430,0.000490,0.148382
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
154435,PANC0203_500x_D13,YWHAG_YWHAZ,-0.060269,YWHAG,YWHAZ,0.134027,False,False,0.100951,False,...,0.519832,-0.161219,-0.295246,False,False,False,0.163768,0.833925,0.059699,0.730925
154436,PANC0203_500x_D17,YWHAG_YWHAZ,-0.060836,YWHAG,YWHAZ,0.112183,False,False,-0.008762,False,...,0.430769,-0.052074,-0.164257,False,False,False,0.257877,0.852498,0.094789,0.956570
154437,PANC1,YWHAG_YWHAZ,-0.648744,YWHAG,YWHAZ,0.135612,False,False,-0.342389,False,...,0.065169,-0.306355,-0.441967,False,False,False,0.040690,0.570509,0.018060,0.739177
154438,SNU410,YWHAG_YWHAZ,0.219989,YWHAG,YWHAZ,-0.030061,False,False,0.060810,False,...,0.431677,0.250049,0.189240,False,False,False,0.889631,0.998589,0.748543,0.998324


In [42]:
melted_pairs.to_csv(os.path.join(LIB_DIRECTORY, "RASCombinatoryEffectMelted.csv"))


In [43]:
## Upload to taiga
import os
from taigapy import create_taiga_client_v3
from taigapy.client_v3 import UploadedFile, LocalFormat, TaigaReference
tc = create_taiga_client_v3()
version = tc.update_dataset(TAIGA_DATASET,  
                            'Uploading the RASCombinatoryEffectMelted',
                            additions=[
    UploadedFile('RASCombinatoryEffectMelted', 
                 local_path=os.path.join(LIB_DIRECTORY, "RASCombinatoryEffectMelted.csv"), 
                 format=LocalFormat.CSV_TABLE)
        
]
)

Created temporary upload session 9e55f136890840c1a0f10d9a9cf3d4d5
Uploading /home/ubuntu/paralog/ras_screens/analysis/RASScreen07212025/RASCombinatoryEffectMelted.csv to S3
Finished uploading /home/ubuntu/paralog/ras_screens/analysis/RASScreen07212025/RASCombinatoryEffectMelted.csv to S3
Completed uploading /home/ubuntu/paralog/ras_screens/analysis/RASScreen07212025/RASCombinatoryEffectMelted.csv to S3
Added /home/ubuntu/paralog/ras_screens/analysis/RASScreen07212025/RASCombinatoryEffectMelted.csv to upload session 9e55f136890840c1a0f10d9a9cf3d4d5
Dataset created. Access it directly with this url: https://cds.team/taiga/dataset/rasscreen07212025-0b0e/23



In [44]:
melted_pairs

,ScreenName,Pairs,ChronosPairs,GeneA,GeneB,ChronosGeneA,IsGeneAEssential,IsGeneAnonessential,ChronosGeneB,IsGeneBEssential,...,minABfdr,CombinatoryScore,SynergyScore,IsEssentialGeneA,IsEssentialGeneB,IncludingEssential,CombinatoryScorePVal,CombinatoryScoreFdr,SynergyScorePVal,SynergyScoreFdr
0,BXPC3,AKT1_AKT2,-0.542547,AKT1,AKT2,-0.173663,False,False,-0.004845,False,...,0.211245,-0.368883,-0.364038,False,False,False,0.016620,0.330214,0.017010,0.746453
1,COLO205,AKT1_AKT2,-0.602124,AKT1,AKT2,-0.002584,False,False,-0.040360,False,...,0.278071,-0.561764,-0.559180,False,False,False,0.050349,0.985395,0.050789,0.851084
2,COLO678,AKT1_AKT2,-0.129872,AKT1,AKT2,-0.098261,False,False,-0.088108,False,...,0.384455,-0.031611,0.056497,False,False,False,0.234268,0.797977,0.399736,1.000000
3,HCT15,AKT1_AKT2,-0.837725,AKT1,AKT2,-0.256233,False,False,0.135923,False,...,0.129046,-0.581492,-0.717415,False,False,False,0.005790,0.245689,0.003440,0.625149
4,HPAFII,AKT1_AKT2,-1.037252,AKT1,AKT2,-0.204400,False,False,-0.148029,False,...,0.125028,-0.832852,-0.684822,False,False,False,0.000010,0.001430,0.000490,0.148382
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
154435,PANC0203_500x_D13,YWHAG_YWHAZ,-0.060269,YWHAG,YWHAZ,0.134027,False,False,0.100951,False,...,0.519832,-0.161219,-0.295246,False,False,False,0.163768,0.833925,0.059699,0.730925
154436,PANC0203_500x_D17,YWHAG_YWHAZ,-0.060836,YWHAG,YWHAZ,0.112183,False,False,-0.008762,False,...,0.430769,-0.052074,-0.164257,False,False,False,0.257877,0.852498,0.094789,0.956570
154437,PANC1,YWHAG_YWHAZ,-0.648744,YWHAG,YWHAZ,0.135612,False,False,-0.342389,False,...,0.065169,-0.306355,-0.441967,False,False,False,0.040690,0.570509,0.018060,0.739177
154438,SNU410,YWHAG_YWHAZ,0.219989,YWHAG,YWHAZ,-0.030061,False,False,0.060810,False,...,0.431677,0.250049,0.189240,False,False,False,0.889631,0.998589,0.748543,0.998324


In [45]:
# Convert melted_df back to wide format
wide_df = melted_pairs.pivot(index='ScreenName', columns='Pairs', values='CombinatoryScore')
wide_df


Pairs,AKT1_AKT2,AKT1_AKT3,AKT1_ALK,AKT1_ARAF,AKT1_BRAF,AKT1_CCND1,AKT1_CCND2,AKT1_CCND3,AKT1_CCNE1,AKT1_CCNE2,...,WWTR1_YAP1,WWTR1_YWHAE,WWTR1_YWHAG,WWTR1_YWHAZ,YAP1_YWHAE,YAP1_YWHAG,YAP1_YWHAZ,YWHAE_YWHAG,YWHAE_YWHAZ,YWHAG_YWHAZ
ScreenName,,,,,,,,,,,,,,,,,,,,,
BXPC3,-0.368883,0.146203,0.010047,0.190517,-0.193217,0.326402,-0.036141,-0.089855,0.126666,0.030803,...,-0.461239,0.120458,0.097665,0.040026,0.401319,0.093640,0.113481,0.326360,-0.079914,-0.110581
COLO205,-0.561764,-0.037920,0.111553,-0.133125,-0.272986,-0.323484,0.084351,-0.478853,0.005624,-0.066960,...,0.199831,0.137309,-0.156043,-0.064506,0.235305,-0.072303,-0.167483,0.189840,-0.043318,-0.422876
COLO678,-0.031611,-0.036938,0.320284,0.307260,-0.025871,0.767561,0.214808,0.222018,0.428822,0.304439,...,0.125347,0.772794,0.525254,0.370921,-0.020140,-0.196893,-0.164260,0.588268,0.394706,0.278939
HCT15,-0.581492,0.102403,-0.061002,0.184644,-0.022517,-0.965629,0.059236,-0.358559,0.137728,0.094716,...,-0.240080,0.083411,-0.265069,-0.283801,0.236444,-0.154552,0.093167,0.295793,0.053886,-0.329579
HPAFII,-0.832852,0.220810,0.252447,0.217967,0.111878,0.256995,0.270954,-0.067030,0.232373,0.250166,...,-0.186278,0.182173,-0.139880,-0.173128,0.170250,0.312099,-0.057205,0.215806,-0.504263,-0.583272
IPC298,0.245321,-0.141564,0.329160,0.433528,0.308685,0.093506,0.320362,-0.183729,0.184393,0.206842,...,-0.948258,0.037285,-0.009000,-0.194524,0.069430,0.087005,0.038780,0.096958,-0.654198,-0.137695
KP363T,-0.173598,0.112357,0.147300,0.125680,-0.024998,0.833976,0.176708,0.237066,0.368024,0.298392,...,-0.347002,-0.345293,-0.256474,0.106453,0.621905,0.694963,0.476454,0.401579,0.475302,-0.031062
PACADD165,0.169614,0.146346,0.420355,0.186126,0.069636,0.704380,0.220075,0.160353,0.326948,0.391285,...,-0.602802,0.189219,0.117542,-0.053902,0.173094,0.185423,-0.039765,0.193194,-0.099747,-0.018170
PACADD188,-0.147560,-0.233068,0.241891,-0.092374,0.152016,-0.185579,-0.124525,-0.072253,-0.112415,-0.004400,...,-0.491826,-0.163907,0.161715,-0.032833,0.116096,-0.009446,0.031237,0.210347,0.244595,0.080296


In [46]:
wide_df.to_csv(os.path.join(LIB_DIRECTORY, "RASCombinatoryEffect.csv"))